<a href="https://colab.research.google.com/github/samikshyasanskruti/Deep-Learning/blob/main/Keras_Implementation_of_SimpleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Libraries

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape,Flatten, Input,Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Hyperparameters

latent_dim = 100
img_shape = (28, 28, 1)
batch_size = 128
epochs = 20
sample_interval = 1000

In [3]:
# Load and preprocess MNIST data

(X_train,_), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.0 # Normalize images to [-1, 1]
X_train = np.expand_dims(X_train, axis=3)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
# Generator Network

def build_generator():
  model = Sequential()
  model.add(Dense(128, input_dim=100))
  model.add(LeakyReLU(0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(0.2))
  model.add(Dense(512))
  model.add(LeakyReLU(0.2))
  model.add(Dense(1024))
  model.add(LeakyReLU(0.2))
  model.add(Dense(784, activation='tanh')) # Output image vector

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(noise, img)

In [5]:
# Discriminator Network

from tensorflow.keras.layers import Dropout, Flatten

def build_discriminator():
  model = Sequential()
  # Change input_dim to 784 to match the flattened image size
  model.add(Dense(1024, input_dim=784))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.3))
  model.add(Dense(512))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.3))
  model.add(Dense(256))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.3))
  model.add(Dense(256))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.3))
  model.add(Dense(128))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.3))
  model.add(Dense(1, activation='sigmoid')) # Probability output

  # The input to the discriminator should be the flattened image
  img = Input(shape=(784,))
  validity = model(img)

  return Model(img, validity)

In [6]:
# Build and compile discriminator

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=Adam(0.0002, 0.5),metrics=['accuracy'])

# Build generator

generator = build_generator()

# Combined GAN model (generator --> discriminator)

z = Input(shape=(latent_dim,))
img = generator(z)
# The discriminator expects a flattened image, so no reshape is needed here as the generator already outputs a flattened image of shape 784.
discriminator.trainable = False # Freeze discriminator during generator training
validity = discriminator(img) # Pass the flattened image directly to the discriminator
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Training function

def train_gan():
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # ---------------------
        # Train Discriminator
        # ---------------------
        # Select random real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Flatten the real images before passing them to the discriminator
        real_imgs = real_imgs.reshape(batch_size, 784)

        # Generate fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        # d_loss_real and d_loss_fake are lists: [loss, accuracy]
        d_loss = 0.5* np.add(d_loss_real, d_loss_fake)

        # ---------------------
        # Train Generator
        # ---------------------
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, valid) # g_loss is a scalar

        # Print progress
        if epoch % sample_interval == 0:
           # Access d_loss elements correctly and handle scalar g_loss
           print(f"{epoch} [D loss: {d_loss[0]} | D accuracy:{100*d_loss[1]}] [G loss: {g_loss}]")

           sample_images(epoch)

# Generate and save sample images
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r* c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5* gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            # Reshape the generated image for displaying
            axs[i,j].imshow(gen_imgs[cnt].reshape(28, 28), cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(f"mnist_{epoch}.png")
    plt.close()

In [8]:
# Start training
train_gan()

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step  


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.7657419443130493 | D accuracy:48.2421875] [G loss: 0.6986088752746582]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
